<center><h1><strong>Diseño de una Montaña Rusa</strong></h1></center>  
<center><img src="Montaña Rusa.jpg" width=450 height=270></center>  

### **Sección: 305C1**
### **Estudiantes:**
Manuel Nava 30.822.007  
Juan Wu 30.391.117  
Enrique Gonzalez 31.456.812

<p align="left">Una empresa de parques de atracciones está diseñando una nueva montaña 
rusa y necesita que la trayectoria de la vía sea suave y continúe en todo el recorrido. 
Para esto, solicitan que se realice un modelo matemático que utilice un método de 
trazador cúbico sujeto. Además, requieren que el análisis de la estabilidad 
estructural de la vía se haga utilizando un polinomio de mínimos cuadrados para 
ajustar los datos experimentales obtenidos de las pruebas de tensión y compresión 
del material. El diseño también debe incluir el uso de polinomios ortogonales para 
optimizar la forma de ciertos tramos de la vía y mejorar la experiencia del usuario. 
Finalmente, se necesita resolver un sistema de ecuaciones para determinar las 
fuerzas que actúan en los puntos críticos de la estructura.
</p>

## Paso 1: Método de Trazador Cúbico Sujeto

Obtener los datos desde un archivo "csv"  
  
El trazador cúbico sujeto es una técnica de interpolación que se utiliza para aproximar una función a partir de un conjunto de puntos de control. A diferencia del trazador cúbico natural, el trazador cúbico sujeto incorpora condiciones adicionales en los extremos del intervalo.

$ S_i(x) = a_i + b_i(x - x_i) + c_i(x - x_i)^2 + d_i(x - x_i)^3 $

In [3]:
import csv

def obtenerelarreglodefilas(archivo, filadelarchivo):
    with open(archivo, 'r') as file:
        leo = csv.reader(file)
        rows = list(leo)
        if filadelarchivo > len(rows) or filadelarchivo < 1:
            raise ValueError("Row number out of range")
        return rows[filadelarchivo - 1]
    
archivo = 'datos.csv'
filadelarchivo = 1
xi = obtenerelarreglodefilas(archivo, filadelarchivo)
xi = [float(cadena) for cadena in xi]

def obtengoyi():
    archivo = 'datos.csv'
    filadelarchivo = 2
    yi = obtenerelarreglodefilas(archivo, filadelarchivo)
    yi = [float(cadena) for cadena in yi]
    return yi

print(xi)
yi = obtengoyi()
print(yi)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
[0.5, 0.8, 1.0, 0.9, 1.2, 0.7]


## Paso 2: Polinomio de Mínimos Cuadrados

In [2]:
print(x)

30


## Paso 3: Polinomios Ortogonales

## Paso 4: Resolución de Ecuaciones

<center><h1><strong>¡Gracias!</strong></h1></center>